In [6]:
#-*- coding:utf-8 -*-
from commonTool import *


In [9]:
# 기준일 로딩

bf = open(resourceDir + 'BASIS-DAYS.txt', 'r', encoding='utf-8')

# 0.YYYYMM, 1.FIRST_DAY, 2.SECOND_DAY, 3.PREV_DAY, 4.LAST_DAY, 5.NEXT_DAY
lineText = bf.readline() # 제목

basisDays = []

while True:
    lineText = bf.readline()
    if not lineText:
        break
        
    lineText = lineText.strip()
    if len(lineText) <= 0:
        continue

    dayData = lineText.split('\t')
    basisDays.append([dayData[0], dayData[3], dayData[4]])

bf.close()


In [34]:
scriptPathName = crawlegoScriptPath + 'APPLY_LOGIC.xml'

resultFolder = outputRawPath + 'logicResult' + os.path.sep
mkdir(resultFolder)

# 초기 투자금
inital_cash = 10000000

y = 0
cash = inital_cash
for i in range(3, len(basisDays), 12):
    basis = basisDays[i]
    sellIdx = min(i + 12, len(basisDays) - 1)
    
    parameter = {
        'BASIS_DAY': basis[1],
        'BUY_DAY': basis[2],
        'SELL_DAY': basisDays[sellIdx][2],
        'CASH': str(cash),
        'TOP_N': '20',
        'OUT_PATH': resultFolder + os.path.sep, # sep이 '/' 라면 더할 필요 없음
        'DO_SERVER': '13.124.29.70'
    }

    y += 1
    retCode = runDashScript(scriptPathName, parameter, False)
    
    if retCode != 0:
        print(basis[0], 'ret code', retCode)
        continue

    # 결과 정리
    statPath = resultFolder + basis[1] + '-STAT.txt'
    sf = open(statPath, 'r', encoding='utf-8')
    lineText = sf.readline() # 제목: 매수일, 매도일, R_SUM
    lineText = sf.readline() # 결과
    rec = lineText.split('\t')
    profitLoss = int(rec[2])
    
    newCash = cash + profitLoss
    
    print('\t'.join([str(y), rec[0], rec[1], str(profitLoss), str(newCash), str(profitLoss / cash), str(pow(newCash / inital_cash, 1 / y) - 1)]))
    
    sf.close()
    cash = newCash;
    
    # itemPath = resultFolder + basis[1] + '-RESULT.txt'
    # rf = open(itemPath, 'r', encoding='utf-8')
    
    # rf.close()
    
    # end of for
    
# end of script

1	20140530	20150529	3709550	13709550	0.370955	0.3709549999999999
2	20150529	20160531	934522	14644072	0.06816576765831117	0.21012693549065342
3	20160531	20170531	3803655	18447727	0.25974025530603784	0.22644371226868065
4	20170531	20180531	5589732	24037459	0.30300383348040655	0.2451513366390794
5	20180531	20190531	-3002706	21034753	-0.12491777937093933	0.1603459301751573
6	20190531	20200529	-2570845	18463908	-0.12221892978729058	0.10761102819961099
7	20200529	20210531	14727841	33191749	0.7976556750607726	0.18695123540474912
8	20210531	20210621	1210170	34401919	0.03645996479426258	0.1670053139871166


In [33]:
# 결과만 다시 보기

y = 0
cash = inital_cash

maxV = inital_cash
maxMdd = 0
for i in range(3, len(basisDays), 12):
    y += 1
    basis = basisDays[i]

    # 결과 정리
    statPath = resultFolder + basis[1] + '-STAT.txt'
    sf = open(statPath, 'r', encoding='utf-8')
    lineText = sf.readline() # 제목: 매수일, 매도일, R_SUM
    lineText = sf.readline() # 결과
    rec = lineText.split('\t')
    profitLoss = int(rec[2])
    
    newCash = cash + profitLoss
    
    maxV = max(maxV, newCash)
    if newCash < cash:
        mdd = (newCash - maxV) / maxV
        maxMdd = min(mdd, maxMdd)

    print('\t'.join([str(y), rec[0], rec[1], str(profitLoss), str(newCash),
        str(profitLoss / cash), str(pow(newCash / inital_cash, 1 / y) - 1)]))
    
    sf.close()
    cash = newCash;
    
    # itemPath = resultFolder + basis[1] + '-RESULT.txt'
    # rf = open(itemPath, 'r', encoding='utf-8')
    
    # rf.close()

print('max mdd', maxMdd * -100)
# end of script

1	20140228	20150227	2688974	12688974	0.2688974	0.26889739999999995
2	20150227	20160229	2986901	15675875	0.2353934210914137	0.25203334620129025
3	20160229	20170228	5989551	21665426	0.38208718811549597	0.2939646470310391
4	20170228	20180228	4276450	25941876	0.19738591800595104	0.2691131519682979
5	20180228	20190228	-82693	25859183	-0.003187626060659607	0.20926911187607877
6	20190228	20200228	-312774	25546409	-0.012095277720104304	0.16919866327446598
7	20200228	20210226	11801409	37347818	0.46195960457690943	0.20712481054960485
8	20210226	20210621	16393515	53741333	0.43894170738435107	0.23392440603094666
max mdd 1.524434855829239
